<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq
'''

'\n!pip install pandas==2.2.2 -qq\n!pip install pyxirr -qq\n!pip install ta -qq\n!pip install yfinance==0.2.59 -qq\n'

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-19 14:42:08
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.33 C
CY Investment:  1.60 C
Reserve:  2.44 L
Current:  1.42 C
-------------------
Today PnL: -1.60 L (-1.12%)
Current PnL: -32.83 L (-20.52%)
CY Booked + Current PnL: -17.15 L (-10.72%)
-------------------
Total profit:  1.09 L
Total loss:  -33.92 L
-------------------
Total Booked + Current PnL: 9.83 L (7.39%)
Total Booked PnL: 42.66 L (32.07%)
Curr Year Booked PnL: 15.68 L (11.07%)
Prev Year Booked PnL: 26.98 L (20.29%)
Est FTT:  2.41 C
Est FTT PnL: 99.44 L (70.22%)
Deployed:  1.33 C
Current:  1.42 C
CAGR/XIRR %: 3.24%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,-1.23,1.91,19.28,21.57,19252.0,1875.0,99855.0,397.04,-11.92,56.0,H-SC,5.09,167.0,0.10,0.72,45.72,XR,ATH,FINANCE
18,COALINDIA,-0.12,2.67,14.61,17.67,22569.0,4014.0,154475.0,478.51,23.39,50.0,L-LC,11.48,185.0,0.18,1.11,24.00,XY25,ATH,MINING
77,TTKPRESTIG,-0.82,-28.39,39.77,0.09,28701.0,-28609.0,72168.0,770.00,67.56,36.0,M-SC,9.46,250.0,-1.00,0.52,0.00,OX40N,NTT,DURABLES
36,ICICIGI,-1.02,2.75,16.14,19.34,33187.0,5510.0,205619.0,2252.93,-13.46,60.0,X-MC,2.60,66.0,0.17,1.48,19.13,X40,ATH,INSURANCE
51,MEDANTA,-1.88,-3.89,29.11,24.08,35516.0,-4944.0,122006.0,1486.00,-4.57,53.0,X-SC,10.61,87.0,-0.14,0.88,11.75,XY24,NTT,HEALTHCARE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,LAOPALA,3.85,-38.77,127.16,39.09,102076.0,-50831.0,80274.0,464.00,75.70,57.0,H-SC,14.66,154.0,-0.50,0.58,7.35,AR,NTT,CERAMICS
14,BSOFT,3.18,-29.70,114.49,50.79,113203.0,-41777.0,98876.0,831.70,-9.93,41.0,H-SC,6.79,172.0,-0.37,0.71,15.12,XR,ATH,IT
20,COLPAL,2.46,-16.32,69.10,41.51,152289.0,-42975.0,220390.0,3726.84,1.32,62.0,X-MC,6.21,62.0,-0.28,1.58,7.69,XY25,ATH,FMCG
43,IRCTC,1.47,-23.76,100.60,52.93,207465.0,-64280.0,206228.0,1269.00,-43.63,52.0,H-MC,12.40,119.0,-0.31,1.48,4.13,XY24,BTT,TRAVEL
69,SURYODAY,1.05,-28.51,72.10,23.03,92302.0,-51051.0,128020.0,216.00,40.35,35.0,H-SC,3.32,166.0,-0.55,0.92,27.02,XR,NTT,BANKS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ATULAUTO,-3.75,-13.55,68.38,45.57,116537.0,-26707.0,170425.0,844.0,3694.59,62.0,M-SC,4.10,248.0,-0.23,1.22,28.97,XY24,NTT,AUTO
25,EASEMYTRIP,-3.63,-48.79,184.48,45.70,167054.0,-86261.0,90554.0,26.4,-5.91,75.0,M-SC,21.36,204.0,-0.52,0.65,49.20,XY24,NTT,TRAVEL
19,COFFEEDAY,-3.27,-45.91,150.08,35.27,92181.0,-52128.0,61421.0,80.0,-57.75,47.0,L-SC,14.44,266.0,-0.57,0.44,50.33,XR,NTT,HOTELS
79,VAIBHAVGBL,-3.02,-26.06,117.42,60.77,158694.0,-47624.0,135151.0,521.0,55.92,51.0,H-SC,2.20,160.0,-0.30,0.97,22.53,XR,NTT,JEWELLERY
70,SYMPHONY,-2.90,-36.79,58.14,-0.03,62899.0,-62955.0,108186.0,1306.0,-49.31,34.0,M-SC,10.04,205.0,-1.00,0.78,0.00,OX40N,NTT,DURABLES


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,-1.47,2.32,103.20,107.91,155902.0,3424.0,151068.0,1641.55,-8.17,60.0,M-SC,8.53,220.0,0.02,1.09,13.65,OX40N,ATH,CABLES
77,TTKPRESTIG,-0.82,-28.39,39.77,0.09,28701.0,-28609.0,72168.0,770.00,67.56,36.0,M-SC,9.46,250.0,-1.00,0.52,0.00,OX40N,NTT,DURABLES
72,TATAELXSI,0.20,-29.45,87.15,32.03,81054.0,-38830.0,93005.0,9161.00,-11.79,36.0,H-SC,7.58,158.0,-0.48,0.67,4.17,OX40N,NTT,IT
66,SIS,-1.93,-29.94,73.09,21.26,56855.0,-33244.0,77788.0,528.00,1827.46,33.0,H-SC,4.33,168.0,-0.58,0.56,7.24,OX40N,NTT,MISC
40,INDIGOPNTS,-0.66,-31.23,45.44,0.02,54548.0,-54515.0,120044.0,1408.00,83.04,29.0,M-SC,9.25,222.0,-1.00,0.86,4.42,OX40N,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,-0.12,2.67,14.61,17.67,22569.0,4014.0,154475.0,478.51,23.39,50.0,L-LC,11.48,185.0,0.18,1.11,24.00,XY25,ATH,MINING
26,FINCABLES,-1.47,2.32,103.20,107.91,155902.0,3424.0,151068.0,1641.55,-8.17,60.0,M-SC,8.53,220.0,0.02,1.09,13.65,OX40N,ATH,CABLES
50,MASFIN,-1.23,1.91,19.28,21.57,19252.0,1875.0,99855.0,397.04,-11.92,56.0,H-SC,5.09,167.0,0.10,0.72,45.72,XR,ATH,FINANCE
1,ABB,-2.13,5.62,37.83,45.57,104529.0,14693.0,276312.0,7934.00,-32.08,61.0,H-MC,5.06,125.0,0.14,1.99,22.70,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,-2.13,5.62,37.83,45.57,104529.0,14693.0,276312.0,7934.00,-32.08,61.0,H-MC,5.06,125.0,0.14,1.99,22.70,AR,NTT,ELECTRICAL
50,MASFIN,-1.23,1.91,19.28,21.57,19252.0,1875.0,99855.0,397.04,-11.92,56.0,H-SC,5.09,167.0,0.10,0.72,45.72,XR,ATH,FINANCE
26,FINCABLES,-1.47,2.32,103.20,107.91,155902.0,3424.0,151068.0,1641.55,-8.17,60.0,M-SC,8.53,220.0,0.02,1.09,13.65,OX40N,ATH,CABLES
39,INDIAMART,-0.78,-7.08,122.47,106.72,140359.0,-8729.0,114607.0,4810.62,-58.01,42.0,H-SC,7.88,140.0,-0.06,0.82,14.65,AR,ATH,MISC
85,ZYDUSLIFE,-0.42,-4.44,41.93,35.63,83973.0,-9304.0,200270.0,1286.17,-16.67,51.0,H-MC,5.26,120.0,-0.11,1.44,11.49,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,INFY,0.68,-5.85,42.58,34.24,125441.0,-18288.0,294600.0,1972.00,-25.98,27.0,X-LC,3.90,3.0,-0.15,2.12,1.95,X40,NTT,IT
33,HCLTECH,-0.42,-5.18,30.60,23.83,70194.0,-12531.0,229393.0,1908.19,-0.33,27.0,X-LC,4.31,8.0,-0.18,1.65,10.06,X40,ATH,IT
73,TCS,0.17,-22.62,59.71,23.58,199879.0,-97879.0,334750.0,4311.59,-32.81,27.0,X-LC,4.80,2.0,-0.49,2.41,0.27,X40,ATH,IT
8,AWL,-0.83,-34.86,141.31,57.19,300207.0,-113681.0,212446.0,485.00,-62.73,28.0,X-SC,17.39,81.0,-0.38,1.53,0.00,XY24,NTT,FMCG
13,BERGEPAINT,-0.04,-18.02,47.76,21.14,89016.0,-40958.0,186381.0,680.00,-19.82,30.0,X-MC,12.78,68.0,-0.46,1.34,0.37,XY24,NTT,PAINTS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UNITDSPR,-2.01,2.55,17.77,20.77,42668.0,5968.0,240112.0,1644.00,-9.09,54.0,X-MC,1.17,64.0,0.14,1.73,10.14,X40N,NTT,BREWERIES
21,DABUR,-1.49,-1.66,46.12,43.70,112512.0,-4113.0,243955.0,735.00,-10.79,43.0,X-MC,1.76,70.0,-0.04,1.75,13.14,XY24,BTT,FMCG
65,SIEMENS,-1.94,-16.01,49.44,25.51,77275.0,-29795.0,156300.0,4671.50,31.28,51.0,H-LC,2.00,51.0,-0.39,1.12,15.18,AR,ATH,ELECTRICAL
36,ICICIGI,-1.02,2.75,16.14,19.34,33187.0,5510.0,205619.0,2252.93,-13.46,60.0,X-MC,2.60,66.0,0.17,1.48,19.13,X40,ATH,INSURANCE
34,HINDUNILVR,-1.72,-11.17,27.96,13.67,63847.0,-28713.0,228350.0,2922.00,-34.02,38.0,X-LC,2.88,9.0,-0.45,1.64,7.80,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,-0.87,-31.65,140.16,64.15,227956.0,-75311.0,162640.0,3906.00,-48.50,37.0,X-SC,8.67,85.0,-0.33,1.17,0.00,XY24,BTT,CEMENT
8,AWL,-0.83,-34.86,141.31,57.19,300207.0,-113681.0,212446.0,485.00,-62.73,28.0,X-SC,17.39,81.0,-0.38,1.53,0.00,XY24,NTT,FMCG
12,BATAINDIA,-1.49,-46.81,154.21,35.20,105532.0,-60236.0,68434.0,2096.00,-10.32,34.0,X-SC,21.63,91.0,-0.57,0.49,0.00,X40,NTT,FOOTWEAR
53,PGHH,-0.46,-13.52,52.65,32.01,97221.0,-28864.0,184656.0,17616.87,-41.49,33.0,X-MC,9.30,57.0,-0.30,1.33,0.00,X40,ATH,FMCG
73,TCS,0.17,-22.62,59.71,23.58,199879.0,-97879.0,334750.0,4311.59,-32.81,27.0,X-LC,4.80,2.0,-0.49,2.41,0.27,X40,ATH,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,-1.06,-30.67,106.03,42.84,47788.0,-19936.0,45070.0,424.00,-54.82,49.0,X-SC,18.25,80.0,-0.42,0.32,8.41,XY24,NTT,MISC
12,BATAINDIA,-1.49,-46.81,154.21,35.20,105532.0,-60236.0,68434.0,2096.00,-10.32,34.0,X-SC,21.63,91.0,-0.57,0.49,0.00,X40,NTT,FOOTWEAR
58,RELAXO,-2.23,-52.81,222.77,52.31,152593.0,-76662.0,68498.0,1176.00,-49.76,43.0,X-SC,13.98,92.0,-0.50,0.49,2.69,X40N,NTT,FOOTWEAR
51,MEDANTA,-1.88,-3.89,29.11,24.08,35516.0,-4944.0,122006.0,1486.00,-4.57,53.0,X-SC,10.61,87.0,-0.14,0.88,11.75,XY24,NTT,HEALTHCARE
35,HONAUT,-0.86,-23.61,87.61,43.32,109004.0,-38452.0,124420.0,58357.33,-32.99,37.0,X-SC,11.44,90.0,-0.35,0.89,1.02,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,-1.40,-13.66,37.89,19.06,111286.0,-46457.0,293709.0,452.00,-52.69,51.0,X-LC,18.32,1.0,-0.42,2.11,6.35,X40,NTT,FMCG
73,TCS,0.17,-22.62,59.71,23.58,199879.0,-97879.0,334750.0,4311.59,-32.81,27.0,X-LC,4.80,2.0,-0.49,2.41,0.27,X40,ATH,IT
42,INFY,0.68,-5.85,42.58,34.24,125441.0,-18288.0,294600.0,1972.00,-25.98,27.0,X-LC,3.90,3.0,-0.15,2.12,1.95,X40,NTT,IT
75,TMPV,-0.91,-15.70,58.17,33.33,159763.0,-51151.0,274649.0,600.00,-82.53,60.0,X-LC,3.04,4.0,-0.32,1.97,12.28,XY24,NTT,AUTO
33,HCLTECH,-0.42,-5.18,30.60,23.83,70194.0,-12531.0,229393.0,1908.19,-0.33,27.0,X-LC,4.31,8.0,-0.18,1.65,10.06,X40,ATH,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,-1.25,-31.13,92.96,32.89,49429.0,-24034.0,53172.0,1800.00,-350.00,58.0,L-MC,11.15,258.0,-0.49,0.38,46.43,XR,NTT,BANKS
67,SONACOMS,-0.34,-8.31,51.70,39.09,47952.0,-8409.0,92750.0,804.02,-27.36,63.0,M-MC,7.62,193.0,-0.18,0.67,31.21,AR,ATH,AUTO
50,MASFIN,-1.23,1.91,19.28,21.57,19252.0,1875.0,99855.0,397.04,-11.92,56.0,H-SC,5.09,167.0,0.10,0.72,45.72,XR,ATH,FINANCE
18,COALINDIA,-0.12,2.67,14.61,17.67,22569.0,4014.0,154475.0,478.51,23.39,50.0,L-LC,11.48,185.0,0.18,1.11,24.00,XY25,ATH,MINING
7,ATULAUTO,-3.75,-13.55,68.38,45.57,116537.0,-26707.0,170425.0,844.00,3694.59,62.0,M-SC,4.10,248.0,-0.23,1.22,28.97,XY24,NTT,AUTO


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,EASEMYTRIP,-3.63,-48.79,184.48,45.70,167054.0,-86261.0,90554.0,26.40,-5.91,75.0,M-SC,21.36,204.0,-0.52,0.65,49.20,XY24,NTT,TRAVEL
11,BANDHANBNK,-0.67,-17.94,134.48,92.42,307409.0,-49968.0,228591.0,400.00,96.12,69.0,H-SC,6.67,174.0,-0.16,1.64,28.97,XY24,NTT,BANKS
67,SONACOMS,-0.34,-8.31,51.70,39.09,47952.0,-8409.0,92750.0,804.02,-27.36,63.0,M-MC,7.62,193.0,-0.18,0.67,31.21,AR,ATH,AUTO
1,ABB,-2.13,5.62,37.83,45.57,104529.0,14693.0,276312.0,7934.00,-32.08,61.0,H-MC,5.06,125.0,0.14,1.99,22.70,AR,NTT,ELECTRICAL
7,ATULAUTO,-3.75,-13.55,68.38,45.57,116537.0,-26707.0,170425.0,844.00,3694.59,62.0,M-SC,4.10,248.0,-0.23,1.22,28.97,XY24,NTT,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.71
1,25,45.35
2,50,77.24


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.37
MC    29.89
LC    24.73
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.04
X40      23.99
X40N     14.84
AR        8.87
XY25      8.69
XR        8.57
OX40N     7.14
SR        0.85
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    22.52
H-SC    22.26
X-LC    21.44
X-SC    11.60
M-SC    10.76
H-MC     4.91
M-MC     2.08
H-LC     1.12
L-LC     1.11
M-LC     1.06
L-SC     0.75
L-MC     0.38
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.52,-14.26,52.90
FINANCE,13.11,-14.51,59.57
IT,11.75,-34.20,101.45
MISC,6.93,-34.58,89.63
ELECTRICAL,6.11,-10.29,50.53
PAINTS,5.01,-32.29,51.95
INSURANCE,4.80,-1.40,36.11
PHARMA,4.07,-3.84,36.53
AUTO,3.86,-16.04,60.29


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3361373.0,21
X40,1382717.0,15
AR,1338691.0,10
XR,1297299.0,12
X40N,1057517.0,10
OX40N,798664.0,10
XY25,407475.0,6
SR,300639.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3694399.0,24
M-SC,1419097.0,14
X-MC,1379200.0,15
X-LC,1288836.0,12
X-SC,1189764.0,10
H-MC,395967.0,3
L-SC,186209.0,2
M-LC,144006.0,1
M-MC,97624.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1257783.0      6
           AR         950641.0      5
           XR         832790.0      7
M-SC       XY24       813793.0      6
X-LC       X40        654284.0      6
X-SC       XY24       611467.0      4
X-MC       X40        549632.0      7
           X40N       426112.0      4
X-SC       X40N       399496.0      4
H-SC       OX40N      352546.0      4
M-SC       OX40N      352090.0      5
H-SC       SR         300639.0      2
X-LC       XY24       269337.0      2
           X40N       231909.0      2
H-MC       XY24       207465.0      1
X-MC       XY25       201928.0      2
           XY24       201528.0      2
H-MC       AR         188502.0      2
M-SC       XR         178893.0      2
X-SC       X40        178801.0      2
M-LC       XR         144006.0      1
X-LC       XY25       133306.0      2
L-SC       OX40N       94028.0      1
           XR          92181.0      1
H-LC       AR          77275.0      1
M-SC       AR          74321.0      1
M-MC       XY25        49672.0      1
L-MC       XR          49429.0      1
M-MC       AR          47952.0      1
L-LC       XY25        22569.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 23.0 seconds
